### Solutions

#### Question 1

Write a decorator that can be used to print out how long a function takes to run.

In [11]:
from time import perf_counter

def log_time(function):
    def inner(*args, **kwargs):
        start = perf_counter()
        result = function(*args, **kwargs)
        end = perf_counter()
        print(f'{function.__name__} took {end - start} seconds to calculate.')
        return result
    return inner
                 
@log_time
def add(a,b,c):
    return a + b + c
          
add(1 , 2, 3)

add took 1.4579854905605316e-06 seconds to calculate.


6

#### Question 2

We have several functions in our code that perform some calculations and return a numeric result, possibly `float`, `int` or even `Decimal`.

We actually want to make sure that all results from each of these functions are rounded to some number of digits after the decimal point (precision), and always returned as a `float`.

But every time our program runs, that precision could change. Also, we'd rather not have to change every function we have, since at some point in the future we may want to return `Decimal` objects, and not `floats` - so we want to minimize how much code we would have to change to accomodate all this.

For example, we might a variable in our code that defines the precision, and could be changed any time we run our code:

In [1]:
PRECISION = 2

Suppose we have the following functions already defined:

In [46]:
from decimal import Decimal

PRECISION = 5

def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0
    
def sum_squares(*args):
    return sum(x**2 for x in args)

def avg(*args):
    if len(args) == 0:
        return 0
    
    numbers = [Decimal(x) for x in args]
    sum_ = sum(numbers)
    return sum_ / len(numbers)

# def normalize(function):
#     def inner(*args, **kwargs):
#         result = function(*args, **kwargs)
#         rounded = round(float(result), PRECISION) if type(result) == float else result
#         return rounded
#     return inner

# @normalize
# def perc_diff(x, y):
#     try:
#         return (y-x) / x * 100
#     except ZeroDivisionError:
#         return 0

# perc_diff(2.6 ,7.3)

# Question 4
def normalize(precision):
    def decorator(function):
        def inner(*args, **kwargs):
            result = function(*args, **kwargs)
            rounded = round(float(result), precision) if type(result) == float else result
            return rounded
    return inner

@normalize(5)
def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0
    
perc_diff(1.5, 7.9)

NameError: name 'inner' is not defined

Write a decorator named `normalize` that can be used to decorate these functions to ensure that the result is always returned as a `float` with a precision defined by some global variable `PRECISION`.

#### Question 3

Sometimes we have functions that get called often with the same argument values but take a long time to run.

If those functions are deterministic (i.e. passing the same arguments will always result in the same return value), then we can get a huge performance benefit by implementing a caching mechanism.

This function simulates a long running function:

In [44]:
from time import sleep
from functools import lru_cache
from timeit import timeit

@lru_cache(maxsize = 2)
def add(x, y):
    sleep(2)
    return x + y

timeit('add(1,2)', globals=globals(), number=10)



2.0051234160200693

As you can see the function is deterministic - the result will always be the same for the same arguments.

Use Python's LRU caching decorator to help improve performance when this function is called multiple times with the same arguments.

Then use `timeit` to test how performance is affected.

#### Question 4

This is kind of a "bonus" exercise. It's a follow-up to Question 2.

It's also complicated, so don't worry if you are unable to do this one!

In Question 2, we created a decorator that used a global variable for the precision.

Here, we would rather define a decorator that can take that precision as an argument, i.e. we could do something like this:

```
@normalize(2)
def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0
    
@normalize(4)
def sum_squares(*args):
    return sum(x**2 for x in args)

@normalize(8)
def avg(*args):
    if len(args) == 0:
        return 0
    
    numbers = [Decimal(x) for x in args]
    sum_ = sum(numbers)
    return sum_ / len(numbers)
```

As a hint, remember how we created "partial" functions in a previous exercise?

What we'll want to do here is not write a decorator function directly, but instead write a function that will **create** a decorator function, with the precision captured in the decorator function (which will itself then, be a closure).

Something like this:

```
def normalize(precision):
    def decorator(fn):
        def inner(*args, **kwargs):
            # precision passed to normalize is available here
            return result
        return inner
    return decorator
```